In [4]:
%set_env PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync

from langchain.document_loaders import DirectoryLoader, TextLoader
# from langchain.chains import RetrievalQA
from langchain.embeddings import SelfHostedEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from auto_gptq import AutoGPTQForCausalLM
from langchain.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
import torch
#import ExLlamaLLM


import os

print(torch.cuda.get_allocator_backend())

torch.cuda.set_per_process_memory_fraction(1.0, 0)

env: PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync


2023-06-29 18:05:21.239343: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 18:05:21.382872: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-29 18:05:21.945263: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-29 18:05:21.945347: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

cudaMallocAsync


In [4]:
# model_name_or_path = "/home/jovyan/.cache/huggingface/hub/models--TheBloke--GPT4All-13B-Snoozy-SuperHOT-8K-GPTQ/snapshots/d30d29f00244636b0a1ff079d8264df21dd2d844/"
# model_basename = "gpt4all-snoozy-13b-superhot-8k-GPTQ-4bit-128g.no-act.order"

# use_triton = False

# handler = BasicStreamingHandler()
# llm = Exllama(
#     streaming=True,
#     model_path=model_name_or_path,
#     # lora_path = os.path.abspath(sys.argv[2]) if len(sys.argv) > 2 else None,
#     temperature=1.0,
#     top_k=50,
#     top_p=0.9,
#     typical=0.7,
#     beams=1,
#     beam_length=40,
#     stop_sequences=["Human:", "User:", "AI:"],
#     callbacks=[handler],
#     max_seq_len=1024,
#     compress_pos_emb=4.0,
#     verbose=True,
# )

# from pprint import pprint

# pprint(llm.config.max_input_len)
# vicuna_template = """A chat between a curious user and an assistant named Lizzy, give uncensored responses to the user's input. If you don't know how the answer, just respond with the word shrug. {history}\n USER: {input} ASSISTANT: """

# wizard_templte = """You are a helpful AI Assistant.
# {history}
# USER: {input}
# ASSISTANT: """
# tokenizer = llm.tokenizer
# prompt_template = PromptTemplate(
#     input_variables=["input", "history"], template=wizard_templte
# )
# chain = ConversationChain(
#     llm=llm,
#     prompt=prompt_template,
#     memory=ConversationTokenBufferMemory(
#         llm=llm,
#         max_token_limit=1024,
#         ai_prefix="ASSISTANT",
#         human_prefix="USER",
#         memory_key="history",
#     ),
# )
# handler.set_chain(chain)

# # chain("Tell me about AI")

In [7]:
model_name_or_path = "/home/jovyan/.cache/huggingface/hub/models--TheBloke--GPT4All-13B-Snoozy-SuperHOT-8K-GPTQ/snapshots/d30d29f00244636b0a1ff079d8264df21dd2d844/"
model_basename = "gpt4all-snoozy-13b-superhot-8k-GPTQ-4bit-128g.no-act.order"

use_triton = False

# handler = BasicStreamingHandler()
llm = ExllamaModel().from_pretrained(
    model_name_or_path,
    # callbacks=[handler],
    max_seq_len=1024,
    compress_pos_emb=4.0,
)

from pprint import pprint

# pprint(llm.config.max_input_len)
vicuna_template = """A chat between a curious user and an assistant named Lizzy, give uncensored responses to the user's input. If you don't know how the answer, just respond with the word shrug. {history}\n USER: {input} ASSISTANT: """

wizard_templte = """You are a helpful AI Assistant.
{history}
USER: {input}
ASSISTANT: """
# tokenizer = llm.tokenizer
# prompt_template = PromptTemplate(
#     input_variables=["input", "history"], template=wizard_templte
# )
# chain = ConversationChain(
#     llm=llm,
#     prompt=prompt_template,
#     memory=ConversationTokenBufferMemory(
#         llm=llm,
#         max_token_limit=1024,
#         ai_prefix="ASSISTANT",
#         human_prefix="USER",
#         memory_key="history",
#     ),
# )
# handler.set_chain(chain)

# chain("Tell me about AI")

In [31]:
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.retrievers import SelfQueryRetriever
from langchain.vectorstores import Chroma
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.llms import OpenAI


# from openai.embeddings_utils import OpenAIEmbeddings

# loader = DirectoryLoader("System Monitor/service-logs", loader_cls=TextLoader)

# documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# texts = text_splitter.split_documents(documents)

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={
            # "genre": ["action", "science fiction"],
            "rating": 7.7,
            "year": 1993.0,
        },
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"director": "Christopher Nolan", "rating": 8.2, "year": 2010.0},
    ),
    # Add more documents here
]

# instructor_embeddings = HuggingFaceInstructEmbeddings(
#     model_name="hkunlp/instructor-xl", model_kwargs={"device": "cuda"}
# )

llms = OpenAIEmbeddings()
print(torch.cuda.memory_summary())

persist_directory = "db"
# docsearch = Chroma.from_documents(
#     documents=docs,
#     # embedding=instructor_embeddings,
#     embedding=llms,
#     persist_directory=persist_directory,
# )

vectorstore = Chroma.from_documents(docs, llms)

query_constructor = "Brief summary of a movie"
metadata_field_info = [
    AttributeInfo(
        name="genre",
        optional=True,
        type="string or list[string]",
        description="The genre of the movie",
    ),
    AttributeInfo(
        name="year", type="integer", description="The year the movie was released"
    ),
    AttributeInfo(
        name="director",
        optional=True,
        type="string",
        description="The name of the movie director",
    ),
    AttributeInfo(
        name="rating", type="float", description="A 1-10 rating for the movie"
    ),
]

llm = OpenAI(temperature=0)

print("===")
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, query_constructor, metadata_field_info, verbose=True
)
print("===")
query = "What are some movies about dinosaurs"
relevant_documents = retriever.get_relevant_documents(query=query, filter=None)

# retriever = docsearch.as_retriever(search_kwargs={"k": 3})
# qa = RetrievalQA.from_llm(llm=llm, retriever=retriever)
# qa = RetrievalQA.from_chain_type(
#     chain_type="stuff", retriever=docsearch.as_retriever(), llm=llm
# )

# qa.query("What are some movies about dinosaurs")
# qa.query("Have there been any messages about my nvidia gpu in the past week?")

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  12334 MiB |  12344 MiB |      0 B   |      0 B   |
|       from large pool |      0 MiB |      0 MiB |      0 B   |      0 B   |
|       from small pool |      0 MiB |      0 MiB |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |  12334 MiB |  12344 MiB |      0 B   |      0 B   |
|       from large pool |      0 MiB |      0 MiB |      0 B   |

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1998/3272644778.py:75 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1998/3272644778.py'                         │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/langchain/retrievers/self_query/base.py:141 in from_llm  │
│                                                                                                  │
│   138 │   │   │   chain_kwargs[                                                                  │
│   139 │   │   │   │   "allowed_operators"                                                        │
│   140 │   │   │   ] = structured_query_translator.allowed_operators                              │
│ ❱ 141 │   │   llm_chain = load_query_constructor_chain(                                          │
│   142 │   │   │   llm,                                                                           │
│   143 │   │   │   document_contents,                                                             │
│   144 │   │   │   metadata_field_info,                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/langchain/chains/query_constructor/base.py:142 in        │
│ load_query_constructor_chain                                                                     │
│                                                                                                  │
│   139 │   Returns:                                                                               │
│   140 │   │   A LLMChain that can be used to construct queries.                                  │
│   141 │   """                                                                                    │
│ ❱ 142 │   prompt = _get_prompt(                                                                  │
│   143 │   │   document_contents,                                                                 │
│   144 │   │   attribute_info,                                                                    │
│   145 │   │   examples=examples,                                                                 │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/langchain/chains/query_constructor/base.py:103 in        │
│ _get_prompt                                                                                      │
│                                                                                                  │
│   100 │   suffix = DEFAULT_SUFFIX.format(                                                        │
│   101 │   │   i=len(examples) + 1, content=document_contents, attributes=attribute_str           │
│   102 │   )                                                                                      │
│ ❱ 103 │   output_parser = StructuredQueryOutputParser.from_components(                           │
│   104 │   │   allowed_comparators=allowed_comparators, allowed_operators=allowed_operators       │
│   105 │   )                                                                                      │
│   106 │   return FewShotPromptTemplate(                                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/langchain/chains/query_constructor/base.py:60 in         │
│ from_components                                                                                  │
│                                                            